In [ ]:
'''#per vedere più immagini

# Percorso alla cartella da cui vuoi prendere le 9 immagini
img_dir = os.path.join(dataset_path, 'paired_dataset_art', 'damaged')

# Prende 9 file casuali (assicurati che ci siano almeno 9 immagini!)
img_files = random.sample(os.listdir(img_dir), 9)

# Crea griglia 3x3
fig, axes = plt.subplots(3, 3, figsize=(12, 10))
axes = axes.flatten()

# Mostra le immagini
for i, img_file in enumerate(img_files):
    img_path = os.path.join(img_dir, img_file)
    img = plt.imread(img_path)
    axes[i].imshow(img)
    axes[i].axis('off')
    axes[i].set_title(f"{img_file}")

plt.tight_layout()
plt.show()'''


CODICE AGGIORNATO

In [ ]:
# ✅ Librerie generali
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ✅ Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# ✅ Percorsi base
dataset_path = "/content/drive/MyDrive/proj_cv/dataset"
paired_path = os.path.join(dataset_path, "paired_dataset_art")
unpaired_path = os.path.join(dataset_path, "unpaired_dataset_art")
damaged_path = os.path.join(paired_path, "damaged")
undamaged_path = os.path.join(paired_path, "undamaged")

# ✅ Percorso per immagini preprocessate
output_dir = "/content/drive/MyDrive/proj_cv/processed_images"
os.makedirs(output_dir, exist_ok=True)
processed_damaged_path = os.path.join(output_dir, "damaged")
processed_undamaged_path = os.path.join(output_dir, "undamaged")

os.makedirs(processed_damaged_path, exist_ok=True)
os.makedirs(processed_undamaged_path, exist_ok=True)


# ✅ Visualizza contenuto del dataset
for dirname, _, filenames in os.walk(dataset_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# ✅ Visualizza una griglia di immagini casuali dal dataset
categories = ['damaged', 'undamaged']
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for ax_idx, dataset_type in enumerate([paired_path, unpaired_path]):
    for i, category in enumerate(categories):
        img_dir = os.path.join(dataset_type, category)
        random_img = random.choice(os.listdir(img_dir))
        img_path = os.path.join(img_dir, random_img)

        img = plt.imread(img_path)
        axes[ax_idx, i].imshow(img)
        axes[ax_idx, i].set_title(f"{dataset_type.split('/')[-1]} - {category}")
        axes[ax_idx, i].axis("off")

# Rimuove colonne vuote (3° colonna)
if len(categories) < 3:
    fig.delaxes(axes[0, 2])
    fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()


In [ ]:
# ✅ Estrai 3 immagini danneggiate e cerca quelle restaurate
damaged_images = os.listdir(damaged_path) # listdir ti ritorna la lista DEI NOMI delle immagini
random_damaged = random.sample(damaged_images, 3)

fig, axes = plt.subplots(3, 2, figsize=(10, 10))

for i, damaged_img in enumerate(random_damaged):
    damaged_img_path = os.path.join(damaged_path, damaged_img)
    damaged_img_data = plt.imread(damaged_img_path)

    # Costruisci nome restaurato
    undamaged_img_name = damaged_img.replace('-before', '-after') #sto costruendo il nome delle immagini restuarate
    undamaged_img_path = os.path.join(undamaged_path, undamaged_img_name)

    if os.path.exists(undamaged_img_path):
        undamaged_img_data = plt.imread(undamaged_img_path)
    else:
        undamaged_img_data = None

    axes[i, 0].imshow(damaged_img_data)
    axes[i, 0].set_title(f"Damaged: {damaged_img}")
    axes[i, 0].axis('off')

    if undamaged_img_data is not None:
        axes[i, 1].imshow(undamaged_img_data)
        axes[i, 1].set_title(f"Restored: {undamaged_img_name}")
    else:
        axes[i, 1].set_title("No matching restored image found")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# ✅ Parametri e funzione di preprocessing
IMG_SIZE = (256, 256)

def preprocess_image(img_path):
    """ Load, resize, normalize an image """
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMG_SIZE)
    img = img / 255.0
    return img

# ✅ Esegui preprocessing su una delle immagini danneggiate
damaged_img_path = os.path.join(damaged_path, random_damaged[0])
processed_img = preprocess_image(damaged_img_path)

plt.imshow(processed_img)
plt.title("Preprocessed Image")
plt.axis("off")
plt.show()


In [ ]:
'''import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define dataset paths
paired_path = '/kaggle/input/damaged-and-undamaged-artworks/AI_for_Art_Restoration_2/paired_dataset_art'
damaged_path = '/kaggle/input/damaged-and-undamaged-artworks/AI_for_Art_Restoration_2/paired_dataset_art/damaged'
undamaged_path = '/kaggle/input/damaged-and-undamaged-artworks/AI_for_Art_Restoration_2/paired_dataset_art/undamaged'

# Create output directories for processed images
output_dir = "/kaggle/working/processed_images"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "damaged"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "undamaged"), exist_ok=True)

# Get all images
damaged_images = os.listdir(damaged_path)

print(f"Total Damaged Images: {len(damaged_images)}")'''

In [ ]:
def preprocess_image(image_path):
    """ Load, resize, denoise, and normalize an image """
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Read in BGR format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (256, 256))  # Resize to 256x256 for consistency

    # Denoising: Apply Gaussian Blur
    img = cv2.GaussianBlur(img, (5,5), 0)

    # Normalize pixel values (0-1 range)
    img = img / 255.0
    return img

# Process all images
for damaged_img in damaged_images:
    damaged_img_path = os.path.join(damaged_path, damaged_img)
    processed_damaged = preprocess_image(damaged_img_path)

    # Find corresponding undamaged image
    undamaged_img_name = damaged_img.replace('-before', '-after')
    undamaged_img_path = os.path.join(undamaged_path, undamaged_img_name)

    if os.path.exists(undamaged_img_path):
        processed_undamaged = preprocess_image(undamaged_img_path)
        # Save the processed undamaged image
        cv2.imwrite(os.path.join(output_dir, "undamaged", undamaged_img_name), (processed_undamaged * 255).astype(np.uint8))

    # Save the processed damaged image
    cv2.imwrite(os.path.join(output_dir, "damaged", damaged_img), (processed_damaged * 255).astype(np.uint8))

print("✅ All images processed and saved!")

In [ ]:
# Select 3 random images for visualization
sample_images = np.random.choice(damaged_images, 3, replace=False)

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    original_path = os.path.join(damaged_path, img_name)
    processed_path = os.path.join(output_dir, "damaged", img_name)

    # Load images
    original_img = cv2.imread(original_path)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)  # Convert to RGB

    processed_img = cv2.imread(processed_path)
    processed_img = cv2.cvtColor(processed_img, cv2.COLOR_BGR2RGB)

    # Display images
    axes[i, 0].imshow(original_img)
    axes[i, 0].set_title("Original Damaged Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(processed_img)
    axes[i, 1].set_title("Processed Image")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
'''def apply_histogram_equalization(img):
    # Convert image to grayscale and apply histogram equalization
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    equalized = cv2.equalizeHist(gray)  # Apply histogram equalization
    return equalized

# Select random sample images
sample_images = np.random.choice(damaged_images, 3, replace=False)

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB

    # Apply histogram equalization
    equalized_img = apply_histogram_equalization(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(equalized_img, cmap='gray')  # Grayscale visualization
    axes[i, 1].set_title("Histogram Equalized")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()'''

In [ ]:
'''def apply_clahe(img):
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray)
    return clahe_img

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply CLAHE
    clahe_img = apply_clahe(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(clahe_img, cmap='gray')
    axes[i, 1].set_title("CLAHE Enhanced")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()'''

In [ ]:
def apply_contrast_stretching(img):
    """ Apply Min-Max Contrast Stretching """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    min_val, max_val = np.min(gray), np.max(gray)
    stretched = ((gray - min_val) / (max_val - min_val) * 255).astype(np.uint8)
    return stretched

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply contrast stretching
    stretched_img = apply_contrast_stretching(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(stretched_img, cmap='gray')
    axes[i, 1].set_title("Contrast Stretched")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
'''def apply_contrast_stretching_rgb(img):
    """ Apply Min-Max Contrast Stretching to each RGB channel """
    stretched = np.zeros_like(img)
    for c in range(3):  # R, G, B
        channel = img[:, :, c]
        min_val, max_val = np.min(channel), np.max(channel)
        if max_val > min_val:  # evita divisione per zero
            stretched[:, :, c] = ((channel - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        else:
            stretched[:, :, c] = channel  # se tutti i pixel hanno lo stesso valore
    return stretched

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Applica contrast stretching a colori
    stretched_img = apply_contrast_stretching_rgb(img)

    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(stretched_img)
    axes[i, 1].set_title("Contrast Stretched (RGB)")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()'''


In [ ]:
'''# Preprocess + stretching
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_stretched = apply_contrast_stretching_rgb(img_rgb)

# Convert to grayscale for Canny
gray = cv2.cvtColor(img_stretched, cv2.COLOR_RGB2GRAY)

# Apply Canny
edges = cv2.Canny(gray, threshold1=50, threshold2=150)'''


In [ ]:
'''import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Sample image path (puoi cambiarlo)
img_name = sample_images[0]  # ad es. "painting1-before.jpg"
img_path = os.path.join(output_dir, "damaged", img_name)

# Funzione: contrast stretching su canali RGB
def apply_contrast_stretching_rgb(img):
    stretched = np.zeros_like(img)
    for c in range(3):  # R, G, B
        channel = img[:, :, c]
        min_val, max_val = np.min(channel), np.max(channel)
        if max_val > min_val:
            stretched[:, :, c] = ((channel - min_val) / (max_val - min_val) * 255).astype(np.uint8)
        else:
            stretched[:, :, c] = channel
    return stretched

# STEP 1: Carica immagine originale
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# STEP 2: Applica contrast stretching
img_stretched = apply_contrast_stretching_rgb(img_rgb)

# STEP 3: Converti in grigio e applica Canny
gray = cv2.cvtColor(img_stretched, cv2.COLOR_RGB2GRAY)
edges = cv2.Canny(gray, threshold1=50, threshold2=150)

# Visualizza tutto
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img_rgb)
axes[0].set_title("Original Image")
axes[0].axis("off")

axes[1].imshow(img_stretched)
axes[1].set_title("Contrast Stretched (RGB)")
axes[1].axis("off")

axes[2].imshow(edges, cmap='gray')
axes[2].set_title("Canny Edges")
axes[2].axis("off")

plt.tight_layout()
plt.show()'''


In [ ]:
def apply_sharpening(img):
    """ Apply sharpening filter to enhance edges """
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])  # Sharpening kernel
    sharpened = cv2.filter2D(img, -1, kernel)
    return sharpened

def apply_gaussian_smoothing(img):
    """ Apply Gaussian blur to reduce noise """
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    return blurred

fig, axes = plt.subplots(3, 3, figsize=(15, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply filters
    sharpened_img = apply_sharpening(img)
    blurred_img = apply_gaussian_smoothing(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(sharpened_img)
    axes[i, 1].set_title("Sharpened")
    axes[i, 1].axis('off')

    axes[i, 2].imshow(blurred_img)
    axes[i, 2].set_title("Gaussian Smoothed")
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def apply_sobel(img):
    """ Apply Sobel edge detection in both X and Y directions """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)  # Horizontal edges
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)  # Vertical edges
    sobel_combined = cv2.magnitude(sobelx, sobely)  # Combine both
    return sobel_combined

def apply_canny(img):
    """ Apply Canny edge detection """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 50, 150)  # Tune thresholds as needed
    return edges

fig, axes = plt.subplots(3, 3, figsize=(15, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply edge detection
    sobel_edges = apply_sobel(img)
    canny_edges = apply_canny(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(sobel_edges, cmap='gray')
    axes[i, 1].set_title("Sobel Edge Detection")
    axes[i, 1].axis('off')

    axes[i, 2].imshow(canny_edges, cmap='gray')
    axes[i, 2].set_title("Canny Edge Detection")
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def apply_thresholding(img):
    """ Apply global thresholding """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)  # Global
    return binary

def apply_adaptive_thresholding(img):
    """ Apply adaptive thresholding """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    adaptive = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    return adaptive

fig, axes = plt.subplots(3, 3, figsize=(15, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply thresholding
    global_thresh = apply_thresholding(img)
    adaptive_thresh = apply_adaptive_thresholding(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(global_thresh, cmap='gray')
    axes[i, 1].set_title("Global Thresholding")
    axes[i, 1].axis('off')

    axes[i, 2].imshow(adaptive_thresh, cmap='gray')
    axes[i, 2].set_title("Adaptive Thresholding")
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#NON SEGMENTA BENE
def apply_watershed(img):
    """ Apply Watershed Algorithm for segmentation """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Noise removal
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Sure background
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labeling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all markers so that the unknown region is not 0
    markers = markers + 1
    markers[unknown == 255] = 0

    # Apply watershed
    img_copy = img.copy()
    markers = cv2.watershed(img_copy, markers)
    img_copy[markers == -1] = [255, 0, 0]  # Mark boundaries in red

    return img_copy

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply Watershed
    watershed_img = apply_watershed(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(watershed_img)
    axes[i, 1].set_title("Watershed Segmented")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def apply_kmeans(img, k=3):
    """ Apply K-Means clustering for segmentation """
    img_flat = img.reshape((-1, 3))  # Flatten the image
    img_flat = np.float32(img_flat)

    # Define criteria, number of clusters (K), and apply KMeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    _, labels, centers = cv2.kmeans(img_flat, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert centers back to uint8 and map labels to segmented image
    centers = np.uint8(centers)
    segmented = centers[labels.flatten()]
    segmented = segmented.reshape(img.shape)

    return segmented

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Apply K-Means
    kmeans_img = apply_kmeans(img, k=4)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(kmeans_img)
    axes[i, 1].set_title("K-Means Segmented")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from skimage.feature import hog

def extract_hog_features(img):
    """Extract HOG features"""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    features, hog_image = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return features, hog_image

fig, axes = plt.subplots(3, 3, figsize=(15, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Extract HOG features
    hog_features, hog_img = extract_hog_features(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(hog_img, cmap='gray')
    axes[i, 1].set_title("HOG Features")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from skimage.feature import local_binary_pattern

def extract_lbp_features(img):
    """Extract LBP features"""
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    lbp = local_binary_pattern(gray, P=8, R=1, method="uniform")
    return lbp

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Extract LBP features
    lbp_img = extract_lbp_features(img)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    axes[i, 1].imshow(lbp_img, cmap='gray')
    axes[i, 1].set_title("LBP Features")
    axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#🧪 Confronta l’istogramma del dipinto danneggiato con quello restaurato per capire dove è cambiata la distribuzione cromatica.
#si potranno mostrare tre grafici del color_histogram, uno per quello danneggiato, uno per la ground_truth e uno del modello

def extract_color_histogram(img):
    """Extract color histograms for RGB channels"""
    chans = cv2.split(img)
    colors = ("r", "g", "b")
    plt.figure(figsize=(6, 4))

    for chan, color in zip(chans, colors):
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        plt.plot(hist, color=color)

    plt.title("Color Histogram")
    plt.xlabel("Bins")
    plt.ylabel("# of Pixels")

fig, axes = plt.subplots(3, 2, figsize=(12, 10))

for i, img_name in enumerate(sample_images):
    img_path = os.path.join(output_dir, "damaged", img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Show images
    axes[i, 0].imshow(img)
    axes[i, 0].set_title("Original Image")
    axes[i, 0].axis('off')

    # Plot histogram
    plt.subplot(3, 2, 2 * i + 2)
    extract_color_histogram(img)

plt.tight_layout()
plt.show()

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
import os
import torch
import re
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Enable GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Image transformations (resize, normalize, tensor conversion)
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize to standard dimensions
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

# Custom Dataset class
class ImageRestorationDataset(Dataset):
    def __init__(self, damaged_dir, undamaged_dir, transform=None):
        self.damaged_dir = damaged_dir
        self.undamaged_dir = undamaged_dir
        self.transform = transform
        self.damaged_images = os.listdir(damaged_dir)
        self.undamaged_images = set(os.listdir(undamaged_dir))  # Use a set for fast lookup

    def __len__(self):
        return len(self.damaged_images)

    def __getitem__(self, idx):
        damaged_img_name = self.damaged_images[idx]

        # Find the corresponding undamaged image
        base, ext = os.path.splitext(damaged_img_name)
        undamaged_img_name = re.sub(r'([-_])before$', r'\1after', base) + ext

        undamaged_img_path = os.path.join(self.undamaged_dir, undamaged_img_name)

        # Handle missing files
        if undamaged_img_name not in self.undamaged_images:
            print(f"⚠️ Missing: {undamaged_img_name} → Skipping this sample.")
            return self.__getitem__((idx + 1) % len(self))  # Get another valid sample

        # Load images
        damaged_img_path = os.path.join(self.damaged_dir, damaged_img_name)
        damaged_img = Image.open(damaged_img_path).convert("RGB")
        undamaged_img = Image.open(undamaged_img_path).convert("RGB")

        if self.transform:
            damaged_img = self.transform(damaged_img)
            undamaged_img = self.transform(undamaged_img)

        return damaged_img, undamaged_img


# Initialize dataset
dataset = ImageRestorationDataset(processed_damaged_path, processed_undamaged_path, transform)

# Create DataLoader
batch_size = 8  # Adjust based on GPU memory
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Test DataLoader
damaged_sample, undamaged_sample = next(iter(train_loader))
print(f"Batch shape (Damaged): {damaged_sample.shape}")
print(f"Batch shape (Undamaged): {undamaged_sample.shape}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define Autoencoder
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # 128x128
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),  # 64x64
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),  # 32x32
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),  # 64x64
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # 128x128
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),  # 256x256
            nn.Tanh()  # Output values between -1 and 1 (normalized)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Move model to GPU
model = Autoencoder().to(device)

In [ ]:
criterion = nn.MSELoss()  # Mean Squared Error for pixel-wise loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [ ]:
num_epochs = 10  # Set number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for damaged_img, undamaged_img in train_loader:
        damaged_img, undamaged_img = damaged_img.to(device), undamaged_img.to(device)

        # Forward pass
        restored_img = model(damaged_img)
        loss = criterion(restored_img, undamaged_img)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

In [ ]:
#fa una sorta di inferenza + valutazione ( da approfondire )
def evaluate_model(model, dataloader, criterion):
    model.eval()  # Set to evaluation mode
    total_loss = 0.0
    num_samples = 0

    with torch.no_grad():
        for damaged_img, undamaged_img in dataloader:
            damaged_img, undamaged_img = damaged_img.to(device), undamaged_img.to(device)

            # Generate restored images
            restored_img = model(damaged_img)

            # Compute loss
            loss = criterion(restored_img, undamaged_img)
            total_loss += loss.item() * damaged_img.size(0)
            num_samples += damaged_img.size(0)

    avg_loss = total_loss / num_samples
    return avg_loss

# Compute loss on test/validation set
test_loss = evaluate_model(model, train_loader, criterion)
print(f"📉 Test Loss: {test_loss:.4f}")

In [ ]:
#30 dB è accettabile
#40+ dB è molto buona
#50+ dB è praticamente perfetta

import numpy as np
import torch.nn.functional as F

def psnr(img1, img2):
    mse = F.mse_loss(img1, img2).item()
    if mse == 0:
        return 100  # Perfect reconstruction
    return 20 * np.log10(1.0 / np.sqrt(mse))

def evaluate_psnr(model, dataloader):
    model.eval()
    total_psnr = 0.0
    num_samples = 0

    with torch.no_grad():
        for damaged_img, undamaged_img in dataloader:
            damaged_img, undamaged_img = damaged_img.to(device), undamaged_img.to(device)
            restored_img = model(damaged_img)

            # Compute PSNR
            batch_psnr = sum(psnr(restored_img[i], undamaged_img[i]) for i in range(damaged_img.size(0)))
            total_psnr += batch_psnr
            num_samples += damaged_img.size(0)

    avg_psnr = total_psnr / num_samples
    return avg_psnr

# Compute PSNR on test set
psnr_score = evaluate_psnr(model, train_loader)
print(f"📊 PSNR Score: {psnr_score:.2f} dB")

In [ ]:
import matplotlib.pyplot as plt

# Function to visualize image restoration
def visualize_results(model, dataloader, num_images=5):
    model.eval()
    fig, axes = plt.subplots(num_images, 3, figsize=(10, 5 * num_images))

    with torch.no_grad():
        for i, (damaged_img, undamaged_img) in enumerate(dataloader):
            if i >= num_images:
                break

            # Move to device
            damaged_img, undamaged_img = damaged_img.to(device), undamaged_img.to(device)
            restored_img = model(damaged_img).cpu()

            # Convert tensors to images
            damaged_img = damaged_img.cpu().permute(0, 2, 3, 1).numpy()
            restored_img = restored_img.permute(0, 2, 3, 1).numpy()
            undamaged_img = undamaged_img.cpu().permute(0, 2, 3, 1).numpy()

            # Display results
            axes[i, 0].imshow(damaged_img[0])
            axes[i, 0].set_title("Damaged Image")

            axes[i, 1].imshow(restored_img[0])
            axes[i, 1].set_title("Restored Image")

            axes[i, 2].imshow(undamaged_img[0])
            axes[i, 2].set_title("Ground Truth")

            for ax in axes[i]:
                ax.axis("off")

    plt.tight_layout()
    plt.show()

# Run visualization
visualize_results(model, train_loader)